In [ ]:
# the below code will generate the number of the def files and the morris random parameters values with OAT rule

In [ ]:
#!pip install SALib

In [ ]:
import os, shutil, numpy as np, re, pandas as pd
from SALib.sample import morris

# ==============================
# CONFIG
# ==============================
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
SOURCE_DEFS_DIR = os.path.join(BASE_DIR, "defs")
SA_DEFS_PARENT = os.path.join(BASE_DIR, "SA_defs_morris")
os.makedirs(SA_DEFS_PARENT, exist_ok=True)

RANDOM_SEED = 2025
NUM_TRAJECTORIES = 100   # Set higher for real runs!
NUM_LEVELS = 10

unchanged_files = set([
    "basin_basin.def", "hill_setting1.def", "landuse_setting5.def",
    "stratum_crop.def", "stratum_grass.def", "stratum_shrub.def"
])

files_to_change = [f for f in os.listdir(SOURCE_DEFS_DIR)
                   if f.endswith(".def") and f not in unchanged_files]

SUM_ONE_GROUPS = {
    "soil_loam_9.def": [('clay', 'sand', 'silt')],
    "soil_sand_10.def": [('clay', 'sand', 'silt')],
    "soil_sandy_loam_12.def": [('clay', 'sand', 'silt')],
    "soil_silt_loam_8.def": [('clay', 'sand', 'silt')],
    "soil_silty_clay_loam_3.def": [('clay', 'sand', 'silt')],
    "stratum_cwt_rhododendron_bgc.def": [
        ('epc.leaflitr_fcel','epc.leaflitr_flab','epc.leaflitr_flig'),
        ('epc.frootlitr_fcel','epc.frootlitr_flab','epc.frootlitr_flig'),
        ('epc.deadwood_fcel','epc.deadwood_flig')],
    "stratum_deciduous.def": [
        ('epc.leaflitr_fcel','epc.leaflitr_flab','epc.leaflitr_flig'),
        ('epc.frootlitr_fcel','epc.frootlitr_flab','epc.frootlitr_flig'),
        ('epc.deadwood_fcel','epc.deadwood_flig')],
    "stratum_eastern_white_pine.def": [
        ('epc.leaflitr_fcel','epc.leaflitr_flab','epc.leaflitr_flig'),
        ('epc.frootlitr_fcel','epc.frootlitr_flab','epc.frootlitr_flig'),
        ('epc.deadwood_fcel','epc.deadwood_flig')],
    "stratum_evergreen.def": [
        ('epc.leaflitr_fcel','epc.leaflitr_flab','epc.leaflitr_flig'),
        ('epc.frootlitr_fcel','epc.frootlitr_flab','epc.frootlitr_flig'),
        ('epc.deadwood_fcel','epc.deadwood_flig')],
    "stratum_localdeciduous.def": [
        ('epc.leaflitr_fcel','epc.leaflitr_flab','epc.leaflitr_flig'),
        ('epc.frootlitr_fcel','epc.frootlitr_flab','epc.frootlitr_flig'),
        ('epc.deadwood_fcel','epc.deadwood_flig')]
}

DAY_INT_PARAMS = {"epc.day_leafoff", "epc.day_leafon", "epc.ndays_expand", "epc.ndays_litfall"}

MULT1000_PARAMS = {
    "epc.vpd_close (x1000)",
    "epc.vpd_open (x1000)",
    "gsurf_intercept",
    "snow_light_ext_coef"
}

CN_RANGES = {"broadleaf": (20, 50, 45, 98),
             "pine": (20, 50, 45, 98),
             "alder": (20, 50, 45, 98),
             "grass": (20, 50, 45, 98)}

SOIL_PREFIXES = [
    "soil_loam_9",
    "soil_sand_10",
    "soil_sandy_loam_12",
    "soil_silt_loam_8",
    "soil_silty_clay_loam_3"
]
KSAT0 = "Ksat_0"
KSAT0_V = "Ksat_0_v"

LAPSE_ZONE_PREFIX = "zone_setting2010"
LAPSE_TMAX = "lapse_rate_tmax (= lapse rate)"
LAPSE_TMIN = "lapse_rate_tmin (= lapse rate)"

def detect_veg_type(fn):
    fn = fn.lower()
    if "pine" in fn or "evergreen" in fn: return "pine"
    elif "alder" in fn: return "alder"
    elif "grass" in fn: return "grass"
    return "broadleaf"

def enforce_sum_to_one(params, file_prefix):
    for grp in SUM_ONE_GROUPS.get(file_prefix + ".def", []):
        keys = [f"{file_prefix}_{p}" for p in grp]
        vals = [max(params.get(k, 0), 0) for k in keys]
        s = sum(vals)
        if s == 0:
            vals = [1.0 / len(grp)] * len(grp)
        else:
            vals = [v / s for v in vals]
        decimals = 8
        vals = [round(v, decimals) for v in vals[:-1]]
        last_val = round(1.0 - sum(vals), decimals)
        vals.append(last_val)
        total = sum(vals)
        if abs(total-1.0) > 1e-8:
            vals = [round(v + (1.0-total)/len(vals), decimals) for v in vals]
        for k, v in zip(keys, vals):
            params[k] = v
    return params

def enforce_cn(params, file_prefix):
    veg = detect_veg_type(file_prefix)
    leaf_min, leaf_max, lit_min, lit_max = CN_RANGES[veg]
    leaf_cn_key = f"{file_prefix}_epc.leaf_cn"
    leaflitr_cn_key = f"{file_prefix}_epc.leaflitr_cn"
    if leaf_cn_key in params:
        params[leaf_cn_key] = np.clip(params[leaf_cn_key], leaf_min, leaf_max)
    if leaflitr_cn_key in params:
        min_lit = max(lit_min, params.get(leaf_cn_key, leaf_min)+5)
        params[leaflitr_cn_key] = np.clip(params[leaflitr_cn_key], min_lit, lit_max)
    return params

def enforce_days(params, file_prefix):
    for d in DAY_INT_PARAMS:
        key = f"{file_prefix}_{d}"
        if key in params:
            params[key] = int(round(min(365,max(0,params[key]))))
    l_on = f"{file_prefix}_epc.day_leafon"
    l_off = f"{file_prefix}_epc.day_leafoff"
    if l_on in params and l_off in params:
        if params[l_on] >= params[l_off]:
            params[l_on] = min(params[l_on], 180)
            params[l_off] = max(params[l_on]+30, params[l_off])
            params[l_off] = min(365, params[l_off])
    return params

def enforce_multiple_1000(params):
    for k in params:
        for pname in MULT1000_PARAMS:
            if k.endswith("_" + pname) or k == pname:
                params[k] = int(np.floor(params[k] / 1000)*1000)
    return params

def unify_ksat_pairs(params):
    """
    For every soil, ensure soil_xxx_Ksat_0 == soil_xxx_Ksat_0_v, always.
    """
    for prefix in SOIL_PREFIXES:
        ksat0 = f"{prefix}_{KSAT0}"
        ksat0v = f"{prefix}_{KSAT0_V}"
        if ksat0 in params:
            params[ksat0v] = params[ksat0]
        elif ksat0v in params:
            params[ksat0] = params[ksat0v]
    return params

def unify_zone_lapse(params):
    """
    For each parameter set, set tmin lapse to exactly match tmax lapse.
    """
    tmax_key = f"{LAPSE_ZONE_PREFIX}_{LAPSE_TMAX}"
    tmin_key = f"{LAPSE_ZONE_PREFIX}_{LAPSE_TMIN}"
    if tmax_key in params:
        params[tmin_key] = params[tmax_key]
    elif tmin_key in params:
        params[tmax_key] = params[tmin_key]
    return params

def enforce_all_constraints(params):
    for f in files_to_change:
        file_prefix = f.replace('.def','')
        params = enforce_sum_to_one(params, file_prefix)
        params = enforce_cn(params, file_prefix)
        params = enforce_days(params, file_prefix)
    params = enforce_multiple_1000(params)
    params = unify_ksat_pairs(params)
    params = unify_zone_lapse(params)
    return params

def parse_param_line(line):
    m = re.match(r'^\s*([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)[ \t]+([a-zA-Z0-9_\.]+(?: \(= lapse rate\))?)', line)
    return (float(m.group(1)), m.group(2), line[m.end():]) if m else (None, None, None)

def is_integer_param(file_prefix, param_name):
    if param_name in DAY_INT_PARAMS:
        return True
    if param_name in MULT1000_PARAMS:
        return True
    full_key = f"{file_prefix}_{param_name}"
    return any([full_key.endswith("_" + p) for p in DAY_INT_PARAMS | MULT1000_PARAMS])

# ====== LOAD PARAMETER RANGES & GENERATE SAMPLES
bounds = pd.read_csv(os.path.join(BASE_DIR, "Parameters_range_values.csv"))
param_names, bnds = [], []
for _, r in bounds.iterrows():
    try:
        lo, hi = float(r['lower limit']), float(r['upper limit'])
        if lo < hi:
            param_names.append(r['Parameter name'])
            bnds.append([lo, hi])
    except:
        continue

problem = {'num_vars': len(param_names), 'names': param_names, 'bounds': bnds}
X = morris.sample(problem, N=NUM_TRAJECTORIES, num_levels=NUM_LEVELS, seed=RANDOM_SEED)
raw_df = pd.DataFrame(X, columns=param_names)

# ====== ADJUSTED SAMPLES AND CONSTRAINTS
adjusted_samples = []
for idx, row in raw_df.iterrows():
    params = row.to_dict()
    params = enforce_all_constraints(params)
    for k, v in params.items():
        if "_" in k:
            prefix, param = k.rsplit("_", 1)
            if is_integer_param(prefix, param):
                if param in MULT1000_PARAMS:
                    params[k] = int(np.floor(v / 1000)*1000)
                else:
                    params[k] = int(round(v))
            elif isinstance(v, float):
                params[k] = round(v, 8)
        elif isinstance(v, float):
            params[k] = round(v, 8)
    # Again, enforce both constraints after rounding/typecasting
    params = unify_ksat_pairs(params)
    params = unify_zone_lapse(params)
    adjusted_samples.append(params)

adjusted_df = pd.DataFrame(adjusted_samples, columns=param_names)
adjusted_df.insert(0, "defs_set", [f"defs{i+1}" for i in range(len(adjusted_df))])
adjusted_df.to_csv(os.path.join(BASE_DIR, "defs_parameter_mapping.csv"), index=False)
adjusted_df.drop("defs_set", axis=1).to_csv(os.path.join(BASE_DIR, "morris_parameter_set_full.csv"), index=False)

# ====== WRITE DEFS FILES FOR EACH SAMPLE
def write_defs(sample_params, idx):
    defsdir = os.path.join(SA_DEFS_PARENT, f"defs{idx}")
    os.makedirs(defsdir, exist_ok=True)
    for f in unchanged_files:
        shutil.copy2(os.path.join(SOURCE_DEFS_DIR, f), os.path.join(defsdir, f))
    for f in files_to_change:
        file_prefix = f.replace('.def','')
        lines = open(os.path.join(SOURCE_DEFS_DIR, f)).read().splitlines(True)
        out = []
        for ln in lines:
            val, p, rest = parse_param_line(ln)
            if not p:
                out.append(ln)
                continue
            if p.endswith("default_ID"):
                out.append(ln)
                continue
            k = f"{file_prefix}_{p}"
            if k in sample_params:
                nv = sample_params[k]
                if is_integer_param(file_prefix, p):
                    if p in MULT1000_PARAMS:
                        nv = int(np.floor(nv / 1000) * 1000)
                    out.append(f"{int(nv)} {p}{rest}")
                elif isinstance(nv, float) and float(nv).is_integer():
                    out.append(f"{int(nv)} {p}{rest}")
                else:
                    out.append(f"{nv:.8f} {p}{rest}")
            else:
                out.append(ln)
        open(os.path.join(defsdir, f), 'w').writelines(out)
    return defsdir

maprec = []
for i, row in adjusted_df.iterrows():
    defsdir = write_defs(row.to_dict(), i + 1)
    maprec.append(row.to_dict())
    maprec[-1]['defs_set'] = f"defs{i+1}"
    print("Created:", defsdir)

NUM_DEFS_FOLDERS = len(adjusted_df)

pd.DataFrame(maprec).to_csv(os.path.join(BASE_DIR, "defs_parameter_mapping.csv"), index=False)
print("Mapping file written.")


In [ ]:
import os

BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
SA_DEFS_PARENT = os.path.join(BASE_DIR, "SA_defs_morris")
WORLDFILES_DIR = os.path.join(BASE_DIR, "worldfiles")
os.makedirs(WORLDFILES_DIR, exist_ok=True)

WORLD_HEADER_TEMPLATE = [
    "1 num_basin_files\n",
    "SA_defs_morris/defs{num}/basin_basin.def basin_default_filename\n",
    "1 num_hillslope_files\n",
    "SA_defs_morris/defs{num}/hill_setting1.def hillslope_default_filename\n",
    "1 num_zone_files\n",
    "SA_defs_morris/defs{num}/zone_setting2010.def zone_default_filename\n",
    "5 num_patch_files\n",
    "SA_defs_morris/defs{num}/soil_loam_9.def patch_default_filename\n",
    "SA_defs_morris/defs{num}/soil_sand_10.def patch_default_filename\n",
    "SA_defs_morris/defs{num}/soil_sandy_loam_12.def patch_default_filename\n",
    "SA_defs_morris/defs{num}/soil_silt_loam_8.def patch_default_filename\n",
    "SA_defs_morris/defs{num}/soil_silty_clay_loam_3.def patch_default_filename\n",
    "1 num_landuse_files\n",
    "SA_defs_morris/defs{num}/landuse_setting5.def landuse_default_filename\n",
    "8 num_stratum_files\n",
    "SA_defs_morris/defs{num}/stratum_localdeciduous.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_evergreen.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_crop.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_cwt_rhododendron_bgc.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_deciduous.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_eastern_white_pine.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_grass.def stratum_default_filename\n",
    "SA_defs_morris/defs{num}/stratum_shrub.def stratum_default_filename\n",
    "1 num_base_stations\n",
    "clim/W22 base_station_filename\n"
]

def create_worldfiles(num_files):
    os.makedirs(WORLDFILES_DIR, exist_ok=True)
    for i in range(1, num_files + 1):
        worldfile_name = f"worldfile{i}.hdr"
        worldfile_path = os.path.join(WORLDFILES_DIR, worldfile_name)
        with open(worldfile_path, 'w') as wf:
            for line in WORLD_HEADER_TEMPLATE:
                wf.write(line.format(num=i))
    print(f"Generated {num_files} worldfile.hdr files in {WORLDFILES_DIR}")

if __name__ == "__main__":
    create_worldfiles(NUM_DEFS_FOLDERS)


In [ ]:
NUM_DEFS_FOLDERS = len(adjusted_df)
NUM_DEFS_FOLDERS

In [ ]:
import os
import pandas as pd

# ==============================
# CONFIGURATION
# ==============================
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
RHESSYS_EXE = "./RHESSysEastCoast/rhessysEC.7.2-2025-08-12-15-50-33"
WORLDHDR_DIR = "worldfiles"
OUTPUT_DIR = "SA_output_basin"
JOB_DIR = os.path.join(BASE_DIR, "SA_job")

SLURM_TIME = "90:00:00"
START_DATE = (1930, 1, 1, 1)
END_DATE = (2020, 12, 30, 1)

PARAM_CSV = os.path.join(BASE_DIR, "defs_parameter_mapping.csv")

GW_NAMES = ['gw1', 'gw2', 'gw3']
S_NAMES = ['s1', 's2', 's3']
SV_NAMES = ['sv1', 'sv2']
SVALT_NAMES = ['svalt1', 'svalt2']

def main():
    os.makedirs(JOB_DIR, exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, OUTPUT_DIR), exist_ok=True)
    st_year, st_month, st_day, st_hour = START_DATE
    ed_year, ed_month, ed_day, ed_hour = END_DATE

    param_df = pd.read_csv(PARAM_CSV)
    NUM_DEFS_FOLDERS = len(param_df)

    for i in range(NUM_DEFS_FOLDERS):
        row = param_df.iloc[i]
        gw_vals = [row[name] for name in GW_NAMES]
        s_vals = [row[name] for name in S_NAMES]
        sv_vals = [row[name] for name in SV_NAMES]
        svalt_vals = [row[name] for name in SVALT_NAMES]

        param_str = (
            f"-gw {' '.join(map(str, gw_vals))} "
            f"-s {' '.join(map(str, s_vals))} "
            f"-sv {' '.join(map(str, sv_vals))} "
            f"-svalt {' '.join(map(str, svalt_vals))}"
        )

        worldfile_hdr = f"{WORLDHDR_DIR}/worldfile{i+1}.hdr"
        output_prefix = f"{OUTPUT_DIR}/Salyan{i+1}"
        job_path = os.path.join(JOB_DIR, f"basin{i+1}.job")
        jobname = f"basin{i+1}"
        job_content = f"""#!/bin/bash
#SBATCH --partition=standard
#SBATCH --ntasks=1
#SBATCH --mem=90gb
#SBATCH -t {SLURM_TIME}
#SBATCH --output=/dev/null
#SBATCH --error=basin_rosi.err
#SBATCH --job-name={jobname}
#SBATCH -A lband_group
#SBATCH --mail-type=ALL
cd {BASE_DIR}
{RHESSYS_EXE} -st {st_year} {st_month} {st_day} {st_hour} -ed {ed_year} {ed_month} {ed_day} {ed_hour} -b -g -dynRtZoff -BGC_flag -fracDirectNdep 0.5 -gwtoriparian -t tecfiles/tec_daily_cali.txt -w worldfiles/worldfile -whdr {worldfile_hdr} -r flows/subflow.txt flows/surfflow.txt -pre {output_prefix} {param_str}
"""
        with open(job_path, "w") as f:
            f.write(job_content)
    print(f"Created {NUM_DEFS_FOLDERS} job scripts in {JOB_DIR}")

if __name__ == "__main__":
    main()


In [ ]:
#finally run this code in the terminal 
#bash-4.4$cd /scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/1976_SA_1/salyan/model
#python job_submission_3000.py ./SA_job .job 

#here the job_submission_3000.py will submit only 3000 jobs at a time and then again run the code 

#bash-4.4$cd /scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/1976_SA_1/salyan/model
#python job_submission_6000.py ./SA_job .job 
#this will submit the next 3000 jobs and so on 

###After all the simulations run 

In [ ]:
#Get the number of defs files again usinng this code given below 

In [ ]:
import os

BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
SA_DEFS_PARENT = os.path.join(BASE_DIR, "SA_defs_morris")
NUM_DEFS_FOLDERS = len([d for d in os.listdir(SA_DEFS_PARENT) if d.startswith("defs") and os.path.isdir(os.path.join(SA_DEFS_PARENT, d))])
print("NUM_DEFS_FOLDERS =", NUM_DEFS_FOLDERS)


In [ ]:
#this code will convert the basin results into new csv file for each rin and save the data in the new folder 

In [ ]:
import os
import pandas as pd
from pandas.errors import EmptyDataError

# === CONFIGURATION ===
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
DAILY_OUTDIR = os.path.join(BASE_DIR, "SA_output_basin")
CSV_OUTDIR = os.path.join(BASE_DIR, "SA_output_basin_csv")  # New folder for CSVs
os.makedirs(CSV_OUTDIR, exist_ok=True)

# NUM_DEFS_FOLDERS should already be set before running this code

# Variables of interest
VARIABLES = ['day', 'month', 'year', 'streamflow', 'laiTREE', 'evap', 'trans']

# === PROCESSING LOOP ===
for i in range(1, NUM_DEFS_FOLDERS + 1):
    result_file = os.path.join(DAILY_OUTDIR, f"Salyan{i}_basin.daily")
    out_csv = os.path.join(CSV_OUTDIR, f"basin{i}.csv")
    if not os.path.exists(result_file):
        print(f"WARNING: {result_file} not found! Skipping...")
        continue
    # Read the header line
    try:
        with open(result_file, 'r') as f:
            header = f.readline().strip().split()
    except Exception as e:
        print(f"ERROR: Could not read {result_file}: {e}")
        continue
    # Try to read the data into a DataFrame
    try:
        df = pd.read_csv(
            result_file,
            delim_whitespace=True,
            comment="#",
            skiprows=1,
            header=None,
            names=header
        )
    except EmptyDataError:
        print(f"EmptyDataError: {result_file} is empty or malformed; skipped.")
        continue
    except Exception as e:
        print(f"ERROR reading {result_file}: {e}")
        continue
    if df.empty:
        print(f"WARNING: {result_file} contains no data rows. Skipping...")
        continue
    # Ensure required date components exist and are integers
    date_missing = False
    for col in ['year', 'month', 'day']:
        if col in df.columns:
            try:
                df[col] = df[col].astype(int)
            except Exception as e:
                print(f"ERROR: Could not convert column '{col}' in {result_file}: {e}")
                date_missing = True
                break
        else:
            print(f"WARNING: Column '{col}' missing in {result_file}. Skipping...")
            date_missing = True
            break
    if date_missing:
        continue
    # Build date column
    df['date'] = pd.to_datetime(
        df[['year', 'month', 'day']].astype(str).agg('-'.join, axis=1),
        format="%Y-%m-%d",
        errors='coerce'
    )
    df = df.dropna(subset=['date'])
    if df.empty:
        print(f"WARNING: All dates invalid in {result_file}. Skipping...")
        continue
    # Extract needed columns safely
    missing_vars = [v for v in ['streamflow', 'laiTREE', 'evap', 'trans'] if v not in df.columns]
    if missing_vars:
        print(f"WARNING: Missing variables {missing_vars} in {result_file}. Skipping...")
        continue
    outdf = df[['date', 'streamflow', 'laiTREE', 'evap', 'trans']].copy()
    outdf.loc[:, 'date'] = outdf['date'].dt.strftime("%Y-%m-%d")
    outdf.to_csv(out_csv, index=False)
    print(f"{out_csv} written.")
print("All files processed.")


In [ ]:
#this code below will use the new generated csv files and calcualte the objective functions for each csv file and save the results 

import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# ==== CONFIGURATION ====
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
SIM_DIR = os.path.join(BASE_DIR, "SA_output_basin_csv")  # Processed CSV outputs location
OBS_CSV = os.path.join(BASE_DIR, "W22_runoff.csv")   # Observed runoff location
RESULT_CSV = os.path.join(BASE_DIR, "objective_function_result.csv")  # Output results CSV

# NUM_DEFS_FOLDERS must be defined before running this script!

# === ANALYSIS DATE RANGE ===
START_DATE = "1976-01-01"
END_DATE   = "2020-12-30"

# ---- Objective Functions ----
def calc_nse(sim, obs):
    obs_mean = np.mean(obs)
    return 1 - np.sum((sim - obs)**2) / np.sum((obs - obs_mean)**2)

def calc_rmse(sim, obs):
    return np.sqrt(mean_squared_error(obs, sim))

def calc_r2(sim, obs):
    return r2_score(obs, sim)

def calc_lognse(sim, obs):
    mask = (sim > 0) & (obs > 0)
    if not np.any(mask):
        return np.nan
    logsim = np.log(sim[mask])
    logobs = np.log(obs[mask])
    return 1 - np.sum((logsim - logobs) ** 2) / np.sum((logobs - logobs.mean()) ** 2)

# ---- LOAD OBSERVED DATA ----
if not os.path.exists(OBS_CSV):
    raise FileNotFoundError(f"Observed data not found: {OBS_CSV}")
obs_df = pd.read_csv(OBS_CSV, parse_dates=["Date"])
obs_df.rename(columns={"runoff(mm/day)_815km2": "runoff"}, inplace=True)
obs_df = obs_df[["Date", "runoff"]]
obs_df = obs_df[(obs_df["Date"] >= START_DATE) & (obs_df["Date"] <= END_DATE)]
obs_df.set_index("Date", inplace=True)
years_with_obs = set(obs_df.dropna(subset=["runoff"]).index.year)
print(f"Years with observed data: {sorted(years_with_obs)}")

results = []

# ---- LOOP OVER SIMULATIONS ----
for i in range(1, NUM_DEFS_FOLDERS + 1):
    job_name = f"basin{i}"
    defs_name = f"defs{i}"
    sim_csv = os.path.join(SIM_DIR, f"basin{i}.csv")  # Correct file name!
    if not os.path.exists(sim_csv):
        print(f"WARNING: {sim_csv} not found; skipping.")
        continue
    try:
        sim_df = pd.read_csv(sim_csv, parse_dates=["date"])
    except Exception as e:
        print(f"ERROR reading {sim_csv}: {e}")
        continue
    sim_df = sim_df[(sim_df["date"] >= START_DATE) & (sim_df["date"] <= END_DATE)]
    sim_df.set_index("date", inplace=True)
    merged = pd.merge(sim_df, obs_df, left_index=True, right_index=True, how='inner')
    if merged.empty:
        print(f"No overlap for {job_name}; skipping.")
        continue
    merged = merged[merged.index.year.isin(years_with_obs)]
    n_before = len(merged)
    merged = merged.dropna(subset=["streamflow", "runoff"])
    dropped_n = n_before - len(merged)
    if dropped_n > 0:
        print(f"{job_name}: Dropped {dropped_n} rows with NaN or no obs year.")
    if merged.empty:
        print(f"{job_name}: No valid rows remaining; skipping.")
        continue
    sim = merged["streamflow"].to_numpy()
    obs = merged["runoff"].to_numpy()
    nse = calc_nse(sim, obs)
    rmse = calc_rmse(sim, obs)
    r2 = calc_r2(sim, obs)
    lognse = calc_lognse(sim, obs)
    results.append({
        "job": job_name,
        "defs": defs_name,
        "NSE": nse,
        "RMSE": rmse,
        "R2": r2,
        "logNSE": lognse
    })
    print(f"{job_name}: NSE={nse:.3f}, RMSE={rmse:.3f}, R2={r2:.3f}, logNSE={lognse:.3f}")

# ---- SAVE RESULTS ----
df = pd.DataFrame(results)
df.to_csv(RESULT_CSV, index=False)
print(f"\nSaved results to {RESULT_CSV}")


In this LAI-based optimization, the MODIS LAI values are first divided by 10 to convert them to the correct physical scale before comparison with the RHESSys simulated LAI. The R² and RMSE calculations are performed only on the dates where both MODIS LAI and RHESSys LAI values are available. Because MODIS LAI is not observed daily (due to 8-day compositing, cloud contamination, and missing retrievals), the code identifies and uses only the shared overlapping dates between both datasets and removes any missing values.

If no overlapping dates exist for a given simulation, it is skipped.

This approach ensures that the comparison is accurate, unbiased, and based solely on valid LAI observations, while correctly accounting for the MODIS LAI scaling.

In [ ]:
# This code will generate the optimization function using LAI for the dates that have MODIS LAI

import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# ==== CONFIGURATION ====
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
SIM_DIR = os.path.join(BASE_DIR, "SA_output_basin_csv")  
MODIS_LAI_CSV = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/LAI_w22.csv"
RESULT_CSV = os.path.join(BASE_DIR, "objective_function_result_LAI.csv") 

# NUM_DEFS_FOLDERS must already be defined

# === ANALYSIS DATE RANGE ===
START_DATE = "2010-01-01"
END_DATE   = "2012-12-31"

# ---- Objective Functions ----
def calc_rmse(sim, obs):
    return np.sqrt(mean_squared_error(obs, sim))

def calc_r2(sim, obs):
    return r2_score(obs, sim)

# ---- LOAD MODIS LAI DATA ----
if not os.path.exists(MODIS_LAI_CSV):
    raise FileNotFoundError(f"MODIS LAI data not found: {MODIS_LAI_CSV}")

modis_df = pd.read_csv(MODIS_LAI_CSV)

if 'Date' not in modis_df.columns or 'Lai' not in modis_df.columns:
    raise ValueError(f"MODIS LAI CSV must contain 'Date' and 'Lai' columns. Found: {modis_df.columns.tolist()}")

modis_df['date'] = pd.to_datetime(modis_df['Date'], format='%m/%d/%Y', errors='coerce')
modis_df = modis_df.dropna(subset=['date'])
modis_df = modis_df[['date', 'Lai']]

# ✅ Divide MODIS LAI by 10 to convert to correct scale
modis_df['Lai'] = modis_df['Lai'] / 10.0

modis_df = modis_df[(modis_df['date'] >= START_DATE) & (modis_df['date'] <= END_DATE)]
modis_df.set_index('date', inplace=True)

results = []

# ---- LOOP OVER SIMULATIONS ----
for i in range(1, NUM_DEFS_FOLDERS + 1):

    job_name = f"basin{i}"
    defs_name = f"defs{i}"
    sim_csv = os.path.join(SIM_DIR, f"basin{i}.csv")

    if not os.path.exists(sim_csv):
        print(f"WARNING: {sim_csv} not found; skipping.")
        continue

    try:
        sim_df = pd.read_csv(sim_csv, parse_dates=["date"])
    except Exception as e:
        print(f"ERROR reading {sim_csv}: {e}")
        continue

    sim_df = sim_df[(sim_df["date"] >= START_DATE) & (sim_df["date"] <= END_DATE)]
    sim_df.set_index("date", inplace=True)

    if "laiTREE" not in sim_df.columns:
        print(f"{job_name}: 'laiTREE' missing; skipping.")
        continue

    # ---- Match LAI Dates ----
    shared_dates = sim_df.index.intersection(modis_df.index)
    sim_lai = sim_df.loc[shared_dates, "laiTREE"].dropna()
    modis_lai = modis_df.loc[shared_dates, "Lai"].dropna()

    final_dates = sim_lai.index.intersection(modis_lai.index)
    sim_lai = sim_lai.loc[final_dates]
    modis_lai = modis_lai.loc[final_dates]

    if len(sim_lai) == 0:
        print(f"{job_name}: No overlapping LAI dates; skipping.")
        continue

    # ---- Compute Optimization Metrics ----
    rmse = calc_rmse(sim_lai, modis_lai)
    r2 = calc_r2(sim_lai, modis_lai)

    results.append({
        "job": job_name,
        "defs": defs_name,
        "RMSE_LAI": rmse,
        "R2_LAI": r2,
        "n_dates": len(final_dates)
    })

    print(f"{job_name}: RMSE_LAI={rmse:.3f}, R2_LAI={r2:.3f}, n_dates={len(final_dates)}")

# ---- SAVE RESULTS ----
df = pd.DataFrame(results)
df.to_csv(RESULT_CSV, index=False)

print(f"\nSaved results to {RESULT_CSV}")


In [ ]:
# this code below check if the simulation has nse value greater than the defined value 

In [ ]:
# --- Filter and show models with NSE > threshold ---
NSE_THRESHOLD = 0.5 #for this, this is logNSE

# Use the same result directory as before
RESULT_DIR = BASE_DIR  # Saving in the base model directory

good_nse_df = df[df["logNSE"] > NSE_THRESHOLD].sort_values(by="logNSE", ascending=False)

if not good_nse_df.empty:
    print(f"\nBasin models with NSE > {NSE_THRESHOLD}:")
    print(good_nse_df.to_string(index=False))
    # Save filtered results for later inspection
    good_nse_csv = os.path.join(RESULT_DIR, f"nse_above_{NSE_THRESHOLD}.csv")
    good_nse_df.to_csv(good_nse_csv, index=False)
    print(f"\nSaved filtered results: {good_nse_csv}")
else:
    print(f"\nNo basins have NSE > {NSE_THRESHOLD}.")


### Sensitivity Analysis (Morris-OAT)

In [ ]:
#python code to join annd relates the def parameters files with the obbjective function csv file for all the simulations 


In [ ]:
import os
import pandas as pd

# === CONFIGURATION ===
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
OBJECTIVE_FUNC_CSV = os.path.join(BASE_DIR, "objective_function_result.csv")
DEFS_PARAM_CSV = os.path.join(BASE_DIR, "defs_parameter_mapping.csv")
OUTPUT_CSV = os.path.join(BASE_DIR, "All_Parameters_with_Objective_functions.csv")

# Load CSVs
obj_df = pd.read_csv(OBJECTIVE_FUNC_CSV)
defs_df = pd.read_csv(DEFS_PARAM_CSV)

# Ensure key columns exist and match
if "defs" not in obj_df.columns:
    raise ValueError("'defs' column not found in objective function CSV!")
if "defs_set" not in defs_df.columns:
    raise ValueError("'defs_set' column not found in defs parameter mapping CSV!")

# Merge on the corresponding columns
merged_df = pd.merge(obj_df, defs_df, left_on="defs", right_on="defs_set", how="inner")

# Optionally, drop the duplicate 'defs_set' column if not needed:
merged_df = merged_df.drop(columns=["defs_set"])

# Save new CSV
merged_df.to_csv(OUTPUT_CSV, index=False)
print(f"Merged table written to: {OUTPUT_CSV}")


In [ ]:
#Morris sensitivity analysis 

In [ ]:
import os
import pandas as pd
import numpy as np
from SALib.analyze import morris as morris_analyze
import matplotlib.pyplot as plt

# --- CONFIG ---
BASE_DIR = "/scratch/hjh7hp/Watershed_22_2025_fall/Watershed22_with_new_summer/Sharadha_khola_watershed/SA_full_range/1976/salyan/model"
PARAM_BOUNDS_CSV = os.path.join(BASE_DIR, "Parameters_range_values.csv")
INPUT_CSV = os.path.join(BASE_DIR, "All_Parameters_with_Objective_functions.csv")
RESULTS_DIR = os.path.join(BASE_DIR, "Morris_results")
os.makedirs(RESULTS_DIR, exist_ok=True)
TOP_N = 15
TOP_N_DIR = os.path.join(RESULTS_DIR, f'top_{TOP_N}')
os.makedirs(TOP_N_DIR, exist_ok=True)
METRICS = ['NSE', 'RMSE', 'R2', 'logNSE']
EXCLUDE_COLS = set(['job','defs','NSE','RMSE','R2','logNSE'])

def wrap_label(label, width=12):
    parts = label.split('_')
    wrapped = []
    for part in parts:
        while len(part) > width:
            wrapped.append(part[:width])
            part = part[width:]
        wrapped.append(part)
    return '\n'.join(wrapped)

print("Loading data and filtering parameter columns...")
metrics_df = pd.read_csv(INPUT_CSV)
param_bounds_df = pd.read_csv(PARAM_BOUNDS_CSV)

all_potential_param_cols = []
for col in param_bounds_df['Parameter name']:
    if col not in EXCLUDE_COLS and col in metrics_df.columns:
        try:
            metrics_df[col].astype(float)
            all_potential_param_cols.append(col)
        except Exception:
            print(f"Excluding parameter '{col}' (non-numeric values detected).")
numeric_param_cols = all_potential_param_cols

print(f"Number of numeric parameter columns to use: {len(numeric_param_cols)}")

# Parameter bounds dictionary
used_param_bounds = {
    p: tuple(param_bounds_df[param_bounds_df['Parameter name'] == p][['lower limit', 'upper limit']].values[0])
    for p in numeric_param_cols
}

metrics_df_numeric = metrics_df.copy()
for col in numeric_param_cols:
    metrics_df_numeric[col] = pd.to_numeric(metrics_df_numeric[col], errors='coerce')

valid_mask = metrics_df_numeric[METRICS].notna().any(axis=1)
param_mask = ~metrics_df_numeric[numeric_param_cols].isna().any(axis=1)
metrics_df_cleaned = metrics_df_numeric.loc[valid_mask & param_mask, :]

# CORRECT HERE
n_cleaned = int(metrics_df_cleaned.shape[0])
if n_cleaned == 0:
    print("No usable simulations remain after filtering for numeric parameter columns.")
    exit()
print(f"Rows usable for Morris analysis: {n_cleaned}")

traj_len = int(len(numeric_param_cols)) + 1
n_traj = n_cleaned // traj_len
n_expected = n_traj * traj_len
df_morris = metrics_df_cleaned.iloc[:n_expected, :]
print(f"Analysis uses {df_morris.shape[0]} rows, {n_traj} full trajectories.")

problem = {
    'num_vars': len(numeric_param_cols),
    'names': numeric_param_cols,
    'bounds': [used_param_bounds[p] for p in numeric_param_cols]
}
results_tables = []
top_results_tables = []
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for idx, metric_col in enumerate(METRICS):
    if metric_col not in df_morris.columns:
        print(f"Metric '{metric_col}' not found. Skipping.")
        continue
    mask_metric = ~df_morris[metric_col].isna()
    X = df_morris.loc[mask_metric, numeric_param_cols].values.astype(float)
    Y = df_morris.loc[mask_metric, metric_col].values.astype(float)
    n_metric = X.shape[0]
    traj_metric = n_metric // traj_len
    n_metric_expected = traj_metric * traj_len
    if traj_metric == 0:
        print(f"Not enough complete trajectories for {metric_col}. Skipping.")
        continue
    X = X[:n_metric_expected, :]
    Y = Y[:n_metric_expected]
    num_levels = 4
    results = morris_analyze.analyze(
        problem,
        X,
        Y,
        num_levels=num_levels,
        print_to_console=False
    )
    results_df = pd.DataFrame({
        'Parameter': problem['names'],
        'mu_star': results['mu_star'],
        'sigma': results['sigma'],
        'mu': results['mu'],
        'mu_star_conf': results['mu_star_conf'],
        'metric': metric_col,
        'lower_limit': [problem['bounds'][i][0] for i in range(len(problem['names']))],
        'upper_limit': [problem['bounds'][i][1] for i in range(len(problem['names']))]
    }).sort_values('mu_star', ascending=False).reset_index(drop=True)
    results_tables.append(results_df)
    metric_csv_all = os.path.join(RESULTS_DIR, f'morris_sensitivity_{metric_col}_all.csv')
    results_df.to_csv(metric_csv_all, index=False)
    top_df = results_df.iloc[:TOP_N, :].copy()
    top_results_tables.append(top_df)
    metric_csv_top = os.path.join(TOP_N_DIR, f'top_{TOP_N}_morris_sensitivity_{metric_col}.csv')
    top_df.to_csv(metric_csv_top, index=False)
    plt.figure(figsize=(12, 5))
    params_sorted = [wrap_label(p, 12) for p in top_df['Parameter']]
    mu_star_sorted = top_df['mu_star']
    bars = plt.bar(range(len(params_sorted)), mu_star_sorted, color=colors[idx % len(colors)], edgecolor='black', alpha=0.85)
    plt.ylabel('Morris μ*', fontsize=14, fontweight='bold')
    plt.title(f'Top {TOP_N} Morris Sensitivity ({metric_col})', fontsize=15, fontweight='bold', pad=12)
    plt.xticks(range(len(params_sorted)), params_sorted, rotation=0, ha='center', fontsize=8, color='black')
    for idx_bar, bar in enumerate(bars):
        height = bar.get_height()
        plt.annotate(f"{height:.2f}", xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 4), textcoords="offset points", ha='center', va='bottom', fontsize=11)
    plt.xlabel('Parameter (ranked)', fontsize=15, fontweight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 1])
    plot_path = os.path.join(TOP_N_DIR, f'top_{TOP_N}_morris_{metric_col}.png')
    plt.savefig(plot_path, dpi=300)
    plt.close()
if results_tables:
    results_all = pd.concat(results_tables, ignore_index=True)
    results_all.to_csv(os.path.join(RESULTS_DIR, f'morris_sensitivity_all_metrics_ranked.csv'), index=False)
if top_results_tables:
    top_results_all = pd.concat(top_results_tables, ignore_index=True)
    top_results_all.to_csv(os.path.join(TOP_N_DIR, f'top_{TOP_N}_morris_sensitivity_all_metrics_ranked.csv'), index=False)
print(f"\nTop {TOP_N} Morris sensitivity results and bar plots saved in:\n {TOP_N_DIR}")
